In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Imports from Different Libraries 

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install livelossplot

In [ ]:
!pip install plot_keras_history

In [ ]:
import numpy as np
import pandas as pd
from numpy import mean 
from numpy import std 
import matplotlib.pyplot as plt
import seaborn as sns
 
import pickle 
import keras 
import tensorflow as tf 
import livelossplot 

from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from sklearn.metrics import accuracy_score 
from sklearn.cluster import KMeans 

from sklearn.datasets import make_classification 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB 
from sklearn.manifold import TSNE

from matplotlib import pyplot 

from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor 

from keras.models import Sequential 
from keras.layers import Activation, Dense 
from keras.layers import Dense, Dropout, Flatten 
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from plot_keras_history import show_history, plot_history
from tensorflow.keras import regularizers
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau
from tensorflow.python.client import device_lib

from array import array
#from keras.optimizers import adam_v2 from tensorflow.keras.optimizers 
#import Adam from keras import models

## Import and process the training dataset

In [ ]:
input_file = "../input/smemi309-final-evaluation-challenge-2021/train5slices.csv"
df_train = pd.read_csv(input_file)

In [ ]:
input_file1 = "../input/smemi309-final-evaluation-challenge-2021/test5slices.csv"
df_test = pd.read_csv(input_file1)

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
df_train.shape
#df_test.shape
X = df_train.iloc[:,1:]
y = df_train[["label"]].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
y

In [ ]:
sample_size = X.shape[0]
time_steps  = X.shape[1]
input_dimension = 1

In [ ]:
sample_size

In [ ]:
train_data_reshaped = X.values.reshape(sample_size,time_steps,input_dimension)
print("After reshape train data set shape:\n", train_data_reshaped.shape)
print("1 Sample shape:\n",train_data_reshaped[0].shape)
print("An example sample:\n", train_data_reshaped[0])

In [ ]:
test_data_reshaped = X.values.reshape(X.shape[0],X.shape[1],1)

In [ ]:
  n_timesteps = train_data_reshaped.shape[1] #320
  n_features  = train_data_reshaped.shape[2] #1 

In [ ]:
n_timesteps

In [ ]:
n_features

In [ ]:
print(device_lib.list_local_devices())

In [ ]:
GPU_list = tf.config.list_physical_devices('GPU')

In [ ]:
GPU_list

## T-SNE

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
X_2d = tsne.fit_transform(X)

In [ ]:
df = pd.DataFrame()
df["comp-1"] = X_2d[:,0]
df["comp-2"] = X_2d[:,1]
df["y"] = y

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("cubehelix", 11),
                data=df).set(title="Train_5_Slices data t-SNE projection")

## KERAS

In [ ]:
model = Sequential(name="model_conv1D")
model.add(keras.layers.Input(shape=(n_timesteps,n_features)))
model.add(keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu', kernel_regularizer=l2(0.008), bias_regularizer=l2(0.008), name="Conv1D_1",))
model.add(keras.layers.MaxPooling1D(pool_size=2, name="MaxPooling1D_1"))
model.add(keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu', kernel_regularizer=l2(0.007), bias_regularizer=l2(0.007), name="Conv1D_2",))
model.add(keras.layers.MaxPooling1D(pool_size=2, name="MaxPooling1D_2"))
model.add(keras.layers.Conv1D(filters=128, kernel_size=7, activation='relu', kernel_regularizer=l2(0.0065), bias_regularizer=l2(0.0065), name="Conv1D_3",))
model.add(keras.layers.Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.005), bias_regularizer=l2(0.005)))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax'))
model.summary() #Print model Summary"

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0042)
model.compile(loss= "sparse_categorical_crossentropy" , optimizer=opt, metrics=["accuracy"])

In [ ]:
plot_losses = livelossplot.PlotLossesKeras()

In [ ]:
history = model.fit(train_data_reshaped, y, epochs=1000,
                    batch_size = 512,
                    validation_split=0.2, 
                    verbose=1,
                    shuffle=True, 
                    callbacks=[plot_losses])

In [ ]:
plot_history(history)

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Train_5_Slices model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Train_5_Slices model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

In [ ]:
testX = df_test.values.reshape(264, 320, 1)
prediction_keras = model.predict(testX)
prediction_list_keras = [np.argmax(x) for x in prediction_keras]

In [ ]:
prediction_list_keras

In [ ]:
#Append the items of the length of the list with loop for submission 
s1 = []

for i in range(len(df_test)):
        s1.append(i)        

In [ ]:
my_submission = pd.DataFrame({'id': s1, 'label': prediction_list_keras})
# you could use any filename. We choose submission here
my_submission.to_csv('submission_sequential.csv', index=False)

In [ ]:
my_submission